In [2]:
from fastapi import FastAPI

import pandas as pd
import uvicorn

In [3]:
app = FastAPI()
genreFunction = pd.read_parquet('./data/genre_functions.parquet')

In [120]:

# We apply the str.lower() method so that it does not matter how the word is written.
genreFunction['genres'] = genreFunction['genres'].str.lower()
genreFunction['genres'].unique()

array(['action', 'indie', 'racing', 'strategy', 'rpg', 'simulation',
       'casual', 'adventure', 'sports', 'free to play',
       'massively multiplayer', 'early access',
       'animation &amp; modeling', 'education', 'software training',
       'utilities', 'web publishing', 'design &amp; illustration',
       'audio production', 'video production', 'photo editing'],
      dtype=object)

In [124]:
mostPlayed = genreFunction.groupby('user_id')['playtime_forever'].sum()
mostPlayed.info()

<class 'pandas.core.series.Series'>
Index: 68602 entries, --000-- to zzzmidmiss
Series name: playtime_forever
Non-Null Count  Dtype
--------------  -----
68602 non-null  int64
dtypes: int64(1)
memory usage: 1.0+ MB


In [106]:
# The app.get decorator is used to associate the function with an HTTP GET request.
# '/PlayTimeGenre/{genre}/' is the path for the endpoint, the {genre} is the parameter.
@app.get('/PlayTimeGenre/{genre}/')
def PlayTimeGenre( genre : str ):
    '''
    Find the genre with the most playtime hours.

    Parameters
    ----------
    genre : str
        Desired genre to look for.
    
    Returns
    -------
    int
        Year when the highest number of hours played was recorded for that genre.
    
    Examples
    -------
    >>> PlayTimeGenre('Action')
    2012
    >>> PlayTimeGenre('Indie')
    2006

    '''
    # We look in our DataSet if there is any match with the genre entered.
    genres = genreFunction[genreFunction['genres'] == genre.lower()]

    # We calculate the total playtime for each year in the dataset.
    results = genres.groupby('year')['playtime_forever'].sum()
    
    # Locate the index where the maximum value is. In this case the most played genre.
    idResut = results.idxmax()

    
    
    if genres.empty:
        return f'The genre {genre} does not exist.'

    
    return f'Year with the most playtime hours for {genre}: {idResut}'

In [107]:
test = PlayTimeGenre('RPG')
print(test)

Year with the most playtime hours for RPG: 2011


# Segunda Función

In [148]:
def UserForGenre( genre : str ):
    '''
    Find the user with the most hours played by genre, as well as hours played by year.

    Parameters
    ----------
    genre : str
        Desired genre to look for.
    
    Returns
    -------
    list
        The user name and the amount of hours played by year for that particular genre.
    
    Examples
    -------
    >>> UserForGenre('')
    2012
    >>> UserForGenre('')
    2006
    '''
    
    
    # Look in the DataSet if there is any match with the genre entered.
    genres = genreFunction[genreFunction['genres'] == genre.lower()]

    if genres.empty:
        return f'The genre {genre} does not exist.'

    # Group the DataSet by user ID, then sum the amount of hours played.
    mostPlayed = genres.groupby('user_id')['playtime_forever'].sum().reset_index()
    
    # Locate the index for the player.
    player = genres.loc[genres['playtime_forever'].idxmax()]['user_id']

    # Filter the DataSet with only the player id
    filteredDF = (mostPlayed[mostPlayed['user_id'] == player]).to_dict()

    # Group the DataSet by year and sum the amount of hours played for each year.
    # listHour = filteredDF.groupby(filteredDF['year'])['playtime_forever'].sum()
    print(filteredDF)
    print(f'{player} is the user with the most playtime for the genre {genre}')
    print(f'\nYear\tHours Played')
    for year, hour in filteredDF.items():
        print(f'{year}\t{hour}\n')
    
    return
    
    # print (
    #         {player}' is the user with the most playtime for the genre {genre}
    #             [{"year": year, "Hours Played": hours} for year, hours in zip(mostPlayed["year"], mostPlayed["playtime_forever"])]'
    # )

In [149]:

result = UserForGenre('strategy')

result

AttributeError: 'dict' object has no attribute 'shape'

In [37]:
# Look in the DataSet if there is any match with the genre entered.
genres = genreFunction[genreFunction['genres'] == 'Action']

# Group the DataSet by user ID and year, then sum the amount of hours played.
mostPlayed = genres.groupby(['user_id', 'year'])['playtime_forever'].sum()

# Locate the index for the player.
player = mostPlayed.idxmax()

# Filter the DataSet with only the player id
filteredDF = genreFunction[genreFunction['user_id'] == player]

# Group the DataSet by year and sum the amount of hours played for each year.
listHour = filteredDF.groupby(filteredDF['year'])['playtime_forever'].sum().to_list()

In [96]:
# Sample dataset as a list of dictionaries
dataset = [
    {'ID': 1, 'Name': 'Alice'},
    {'ID': 2, 'Name': 'Bob'},
    {'ID': 3, 'Name': 'Charlie'},
]

# ID to match
target_id = 2

# Filter the dataset based on the target ID
filtered_data = [record for record in dataset if record['ID'] == target_id]

# Display the filtered data
print(filtered_data)


[{'ID': 2, 'Name': 'Bob'}]


In [39]:
for year, hours in listHour.items():
    print(f'{year}-> {hours}')

AttributeError: 'list' object has no attribute 'items'